In [12]:
import pandas as pd
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def get_total_works(driver, author_url):
    driver.get(author_url)
    time.sleep(1)

    if "community/u/" in author_url:
        try:
            work_elements = driver.find_elements(By.CSS_SELECTOR, 'li._1w7ywmw0')
            return len(work_elements)
        except Exception as e:
            print(f"작가 홈 크롤링 오류: {e}")
            return 0

    elif "artistTitle" in author_url:
        try:
            total_text = driver.find_element(By.CSS_SELECTOR, 'span.ComponentHead__count--bRLNZ').text.strip()
            work_count = int(re.search(r'\d+', total_text).group())
            return work_count
        except Exception as e:
            print(f"artistTitle 페이지 크롤링 오류: {e}")
            return 0

    else:
        print("알 수 없는 작가 페이지 형태")
        return 0

def crawl_author_works(title_id, webtoon_url, driver):
    driver.get(webtoon_url)
    time.sleep(1)

    writer_works = 0
    illustrator_works = 0
    writer_illustrator_works = 0
    original_writer_works = 0
    total_works = 0
    is_original_exist = 0

    # ✅ 처음에 전체 작가 수를 파악
    initial_author_blocks = driver.find_elements(By.CSS_SELECTOR, 'span.ContentMetaInfo__category--WwrCp')
    num_authors = len(initial_author_blocks)

    for idx in range(num_authors):
        try:
            # ✅ 웹툰 페이지에서 다시 author_blocks를 새로 찾음
            author_blocks = driver.find_elements(By.CSS_SELECTOR, 'span.ContentMetaInfo__category--WwrCp')
            block = author_blocks[idx]

            role_text = block.text.strip()
            author_name = block.find_element(By.CSS_SELECTOR, 'a').text.strip()
            author_link = block.find_element(By.CSS_SELECTOR, 'a').get_attribute("href")
            print(f"[{title_id}] {role_text} | {author_name} | {author_link}")

            # ✅ 작가 홈에서 작품 수 가져오기
            works_count = get_total_works(driver, author_link)

            # ✅ 크롤링 후 반드시 다시 웹툰 상세 페이지로 돌아오고, 1초 딜레이
            driver.get(webtoon_url)
            time.sleep(1)

            # ✅ 역할별 분류
            if "글/그림" in role_text:
                writer_illustrator_works += works_count
            elif "글" in role_text:
                writer_works += works_count
            elif "그림" in role_text:
                illustrator_works += works_count
            elif "원작" in role_text:
                original_writer_works += works_count
                is_original_exist = 1

            total_works += works_count

        except Exception as e:
            print(f"작가 정보 수집 오류: {e}")
            continue

    # ✅ 결과 출력
    print(f"✅ [title_id: {title_id}] 글: {writer_works}, 그림: {illustrator_works}, 글/그림: {writer_illustrator_works}, 원작: {original_writer_works}, 총합: {total_works}, 원작여부: {is_original_exist}")

    return {
        'title_id': title_id,
        'writer_works': writer_works,
        'illustrator_works': illustrator_works,
        'writer_illustrator_works': writer_illustrator_works,
        'original_writer_works': original_writer_works,
        'total_works': total_works,
        'is_original': is_original_exist
    }

# 🔥 CSV 데이터 읽기
df = pd.read_csv('naver_작가 작품 수, 원작 여부 병합 전_under19.csv', encoding='utf-8')

# 🔥 결과 저장할 폴더 생성
output_dir = "author_data_files"
os.makedirs(output_dir, exist_ok=True)

# 🔥 100개 단위로 청크 처리
chunk_size = 100
for chunk_start in range(0, len(df), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(df))
    df_chunk = df.iloc[chunk_start:chunk_end]
    results = []

    # ✅ Selenium driver 실행
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    for idx, row in df_chunk.iterrows():
        title_id = row['title_id']
        webtoon_url = f"https://comic.naver.com/webtoon/list?titleId={title_id}"
        result = crawl_author_works(title_id, webtoon_url, driver)
        results.append(result)

    driver.quit()

    # ✅ 청크별 CSV로 저장
    df_result = pd.DataFrame(results)
    chunk_filename = f"naver_작가_작품수_세부정보_{chunk_start}_{chunk_end-1}.csv"
    df_result.to_csv(chunk_filename, index=False, encoding='utf-8-sig')
    print(f"✅ 청크 {chunk_start}-{chunk_end-1} 저장 완료: {chunk_filename}")

print("🎉 모든 청크 크롤링 완료!")


[720121] 순끼
∙
글/그림 | 순끼 | https://comic.naver.com/community/u/_vsk4g?lastPage=EpisodeList
✅ [title_id: 720121] 글: 0, 그림: 0, 글/그림: 3, 원작: 0, 총합: 3, 원작여부: 0
[738145] 다홍
∙
글/그림 | 다홍 | https://comic.naver.com/community/u/dahongcomic?lastPage=EpisodeList
✅ [title_id: 738145] 글: 0, 그림: 0, 글/그림: 2, 원작: 0, 총합: 2, 원작여부: 0
[729767] 나윤희
∙
글/그림 | 나윤희 | https://comic.naver.com/artistTitle?id=302692
✅ [title_id: 729767] 글: 0, 그림: 0, 글/그림: 4, 원작: 0, 총합: 4, 원작여부: 0
[718020] 삼촌
∙
글/그림 | 삼촌 | https://comic.naver.com/community/u/_g1ghtc?lastPage=EpisodeList
✅ [title_id: 718020] 글: 0, 그림: 0, 글/그림: 4, 원작: 0, 총합: 4, 원작여부: 0
[796152] 모죠
∙
글/그림 | 모죠 | https://comic.naver.com/community/u/_fbot9?lastPage=EpisodeList
✅ [title_id: 796152] 글: 0, 그림: 0, 글/그림: 6, 원작: 0, 총합: 6, 원작여부: 0
[647946] 허5파6
∙
글/그림 | 허5파6 | https://comic.naver.com/artistTitle?id=257312
✅ [title_id: 647946] 글: 0, 그림: 0, 글/그림: 6, 원작: 0, 총합: 6, 원작여부: 0
[716857] 손하기
∙
글/그림 | 손하기 | https://comic.naver.com/community/u/sonhagi?lastPage=EpisodeList
✅

In [1]:
import pandas as pd
import os

# 1️⃣ 통합할 파일이 들어 있는 폴더 경로
folder_path = "네이버_작가 작품 수, 원작 여부_under19"

# 2️⃣ 통합 결과를 담을 빈 리스트
dfs = []

# 3️⃣ 폴더 내 모든 파일 순회하여 통합
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

# 4️⃣ 모든 파일의 데이터를 하나로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 5️⃣ 최종 파일로 저장 (필요에 따라 경로/파일명 변경)
merged_df.to_csv("네이버_작가_작품수_세부정보_통합.csv", index=False, encoding="utf-8-sig")

print("✅ 통합이 완료되었습니다! 최종 파일: 네이버_작가_작품수_세부정보_통합.csv")


✅ 통합이 완료되었습니다! 최종 파일: 네이버_작가_작품수_세부정보_통합.csv
